In [ ]:
import open3d as o3d
import numpy as np
import copy
from os import listdir
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import time 
from winsound import Beep

treg = o3d.t.pipelines.registration

In [ ]:
def draw_point_clouds(point_clouds, black_point_cloud = None):
    pc_copy = []
    colors = [[1, 0.706, 0], [0, 0.651, 0.929], [0.3, 0.351, 0.529]]
    for idx, x in enumerate(point_clouds):
        temp = copy.deepcopy(x)
        if idx < len(colors):
            temp.paint_uniform_color(colors[idx])
        else:
            temp.paint_uniform_color(np.random.rand(3))
        pc_copy.append(temp)
    
    if black_point_cloud is not None:
        temp = copy.deepcopy(black_point_cloud)
        temp.paint_uniform_color([0,0,0])
        pc_copy.append(temp)
   
    o3d.visualization.draw_geometries(pc_copy)
    

In [ ]:
source_folder = "C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\Daten\\rotated_and_translated\\100000000\\ground_removed_frames\\"

target_path = "C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\LiDAR_Punktwolke-RevA-201455_Vermessungsdaten\\Cleaned Export\\intersection_dirty\\intersection_dirty2_xyzi_down_transformed_compressed_ground_removed.pcd"
target = o3d.io.read_point_cloud(target_path)
voxel_size = 1
target = target.voxel_down_sample(voxel_size)
# target.estimate_normals()

filenames = listdir(source_folder)[:]
frames = [target]
for file in filenames:
    frames.append(o3d.io.read_point_cloud(source_folder + file))

draw_point_clouds(frames)






In [ ]:
def compute_registration(source, target):
    max_correspondence_distance = 0.5
    evaluation = o3d.pipelines.registration.evaluate_registration( source, target, max_correspondence_distance)
    print(evaluation.fitness, evaluation.inlier_rmse, evaluation)

In [ ]:
filenames = listdir(source_folder)

frames = []
for file in filenames:
    frames.append(o3d.io.read_point_cloud(source_folder + file))



In [ ]:
i = 1
for temp_frame in frames:
    print(str(i) + " / " + str(len(frames)) + " ")
    compute_registration(temp_frame, target)
    i += 1

In [ ]:
def transform(point_cloud, transformation):
    temp = copy.deepcopy(point_cloud) 
    return temp.transform(transformation)

In [ ]:
def compute_icp_p2p(source, target):
    max_correspondence_distance = 10
    # return o3d.pipelines.registration.registration_icp( source, target, threshold, np.identity(4), o3d.pipelines.registration.TransformationEstimationPointToPoint(),  o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1.000000e-07, relative_rmse=1.000000e-07,))#max_iteration = 2000,
    return o3d.pipelines.registration.registration_icp( source, target, max_correspondence_distance, np.identity(4), o3d.pipelines.registration.TransformationEstimationPointToPoint())

In [ ]:
def compute_icp_p2plane(source, target):
    max_correspondence_distance = 0.05
    return o3d.pipelines.registration.registration_icp( source, target, max_correspondence_distance, np.identity(4), o3d.pipelines.registration.TransformationEstimationPointToPlane())

In [ ]:
def compute_icp_multi_scale(source, target):
    draw_point_clouds([source], target)
    voxel_sizes = o3d.utility.DoubleVector([0.1, 0.05, 0.025])

    # List of Convergence-Criteria for Multi-Scale ICP:
    criteria_list = [
        treg.ICPConvergenceCriteria(relative_fitness=0.0001, relative_rmse=0.0001, max_iteration=20),
        treg.ICPConvergenceCriteria(0.00001, 0.00001, 15),
        treg.ICPConvergenceCriteria(0.000001, 0.000001, 10)
    ]

    # `max_correspondence_distances` for Multi-Scale ICP (o3d.utility.DoubleVector):
    max_correspondence_distances = o3d.utility.DoubleVector([0.3, 0.14, 0.07])

    # Initial alignment or source to target transform.
    init_source_to_target = o3d.core.Tensor.eye(4, o3d.core.Dtype.Float32)

    # Select the `Estimation Method`, and `Robust Kernel` (for outlier-rejection).
    estimation = treg.TransformationEstimationPointToPlane()

    # Save iteration wise `fitness`, `inlier_rmse`, etc. to analyse and tune result.
    callback_after_iteration = lambda loss_log_map : print("Iteration Index: {}, Scale Index: {}, Scale Iteration Index: {}, Fitness: {}, Inlier RMSE: {},".format(
        loss_log_map["iteration_index"].item(),
        loss_log_map["scale_index"].item(),
        loss_log_map["scale_iteration_index"].item(),
        loss_log_map["fitness"].item(),
        loss_log_map["inlier_rmse"].item()))

    # Setting Verbosity to Debug, helps in fine-tuning the performance.
    # o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)

    s = time.time()

    registration_ms_icp = treg.multi_scale_icp(source, target, 
                                               voxel_sizes,
                                            criteria_list,
                                            max_correspondence_distances,
                                            # init_source_to_target,
                                            # estimation,
                                            # callback_after_iteration
                                            )

    ms_icp_time = time.time() - s
    print("Time taken by Multi-Scale ICP: ", ms_icp_time)
    print("Inlier Fitness: ", registration_ms_icp.fitness)
    print("Inlier RMSE: ", registration_ms_icp.inlier_rmse)
    show_transformation(target, source, registration_ms_icp.transformation)

In [ ]:
# draw_point_clouds([frames[2]], target)
compute_icp_multi_scale(frames[2], target)

In [ ]:
def show_transformation(target, source, transformation):
    draw_point_clouds([target, transform(source, transformation)], source)
    # draw_point_clouds([target, ], source)

def rotate(point_cloud, x_degrees,y_degrees = 0, z_degrees = 0):
    R = point_cloud.get_rotation_matrix_from_xyz(np.pi/180 * np.array([z_degrees, y_degrees,x_degrees]))
    point_cloud.rotate(R)
    return R

def relative_translation_error(translation_vector1, translation_vector2):
    return np.linalg.norm(translation_vector1-translation_vector2)

def relative_rotation_error(rotation, ground_truth_rotation):
    return np.arccos((np.trace(np.matmul(rotation, ground_truth_rotation))-1)/2)

def decompose_transformation_matrix(transformation):
    translation = transformation[0:3, 3]
    rotation = transformation[0:3, 0:3]
    return translation, rotation

In [ ]:
filenames = listdir(source_folder)[5:7]
frames = []
for file in filenames:
    frames.append(o3d.io.read_point_cloud(source_folder + file))
    
df_translation = pd.DataFrame({"frame_number":filenames})
df_rotation = pd.DataFrame({"frame_number":filenames})

# suffix = "relative_fitness=1e-07_rel_rmse=1e-07_p2plane"
suffix = ""
x = 1
display_evaluation = None
# rotations = [0,1,2]
translations = [[0,0,0],[1,0,0],[2,0,0],[3,0,0],[3,3,0],[5,5,0],[10,10,0]][x:x+1]
for translation in translations:
    print(translation)
    translation_error = []
    rotation_error = []
    for idx, frame in enumerate(frames):
        print(str(idx) + " / " + str(len(frames)) + " ")

        temp_frame = copy.deepcopy(frame)
        temp_frame.translate(translation)
        ground_truth_rotation = rotate(temp_frame, 0,0,0)

        # evaluation = compute_icp_p2p(temp_frame, target)
        evaluation = compute_icp_p2p(temp_frame, target)
        display_evaluation = evaluation 

        # draw_point_clouds([target, temp_frame], transform(temp_frame, evaluation.transformation))
        translation_vector, rotation_matrix = decompose_transformation_matrix(evaluation.transformation)
        t_error = relative_translation_error(-np.array(translation), translation_vector)
        r_error = relative_rotation_error(rotation_matrix,ground_truth_rotation.T)
        translation_error.append(t_error)
        rotation_error.append(r_error)
        
        
        show_transformation(target, temp_frame, evaluation.transformation)
        

    df_translation[str(translation)] = translation_error
    df_rotation[str(translation)] = rotation_error

now = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
df_translation.to_csv("C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\Daten\\rotated_and_translated\\100000000\\csv_results\\ground_removed\\"+now+"_local_registration_translation_error_" + suffix + ".csv")
df_rotation.to_csv("C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\Daten\\rotated_and_translated\\100000000\\csv_results\\ground_removed\\"+now+"_local_registration_rotation_error_" + suffix + ".csv")


Beep(300, 200)
Beep(300, 200)
Beep(300, 700)

In [ ]:
df_translation

In [ ]:
df_translation.plot()

In [ ]:
df_rotation

In [ ]:
df_rotation.plot()

In [ ]:


t, R = decompose_transformation_matrix(evaluation.transformation)
print(t, R)


In [ ]:
show_transformation(target, frame, display_evaluation.transformation)

In [ ]:
df
df.to_csv("C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\Daten\\rotated_and_translated\\100000000\\csv_results\\"+str(df_counter)+"_local_registration.csv")


In [ ]:
df.to_csv("C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\Daten\\rotated_and_translated\\100000000\\csv_results\\01_local_registration.csv")

In [ ]:
# [0,0,X]  Höhe

temp = copy.deepcopy(arr[1])
temp.translate([0,0,10])
draw_point_clouds([target,arr[1]], temp)

In [ ]:
df.plot()

Rotation Tests

In [ ]:

temp = copy.deepcopy(arr[0])
x_degree = 10
R = rotate(temp, 359)
temp2 = copy.deepcopy(temp)
temp2.rotate(R.T)
temp2.translate((0,10,0))

print(R)
print(R.T)

draw_point_clouds([arr[0],temp2], temp)




relative_rotation_error(R, np.identity(3))

In [ ]:

temp = copy.deepcopy(arr[0])
x_degree = 10
R = rotate(temp, 359)
temp2 = copy.deepcopy(temp)
temp2.rotate(R.T)
temp2.translate((0,10,0))

print(R)
print(R.T)

draw_point_clouds([arr[0],temp2], temp)


relative_rotation_error(R, np.identity(3))

In [ ]:
rotate(copy.deepcopy(arr[1]), 360,0,0)

In [ ]:
temp = copy.deepcopy(arr[0])
temp2 = copy.deepcopy(temp)

trans_vector = np.array([0,10,2])
# temp.translate(trans_vector)

temp2.translate(trans_vector*(-1))

print(R)

draw_point_clouds([arr[4],temp], )

In [ ]:
relative_translation_error(np.array([0,10,2]),np.array([0,10,2]),)

trans, rot = decompose_transformation_matrix(evaluation.transformation)
relative_rotation_error(rot, )

In [ ]:
rotation = rot
ground_truth_rotation = rot

transformation_base = "C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\Daten\\rotated_and_translated\\100000000\\transformation_matrices_2\\1667908110_100000000_transformation.txt"
transformation_matrix = np.loadtxt(transformation_base, delimiter=" ", dtype=np.float32)
print(transformation_matrix, "\n\n")

trans, rot2 = decompose_transformation_matrix(transformation_matrix)
print(type(rot),"\n\n\n", type(rot2), "\n")


# np.arccos(np.trace(np.matmul(rotation, rot))/2)
np.arccos((np.trace(np.matmul(rotation, rot2))-1)/2)


Global registration

In [ ]:
def preprocess_point_cloud(pcd, voxel_size):
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    pcd_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature( pcd_down, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [ ]:
def compute_global_registration_no_threshold(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    return o3d.pipelines.registration.registration_ransac_based_on_feature_matching( source_down, target_down, source_fpfh, target_fpfh, True, distance_threshold)
    

In [ ]:
def compute_fast_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.5
    # result = o3d.pipelines.registration.registration_fgr_based_on_feature_matching( source_down, target_down, source_fpfh, target_fpfh, o3d.pipelines.registration.FastGlobalRegistrationOption( maximum_correspondence_distance=distance_threshold))
    return o3d.pipelines.registration.registration_fgr_based_on_feature_matching( source_down, target_down, source_fpfh, target_fpfh)

In [ ]:
target_down = None

In [ ]:
voxel_size = 1
suffix = "ransac_no_threshold"


filenames = listdir(source_folder)[6:7]
if target_down is None:
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
print("Target fpfh computation done")

frames = []
frames_fpfh = []
for file in filenames:
    point_cloud = o3d.io.read_point_cloud(source_folder + file)
    source_down, source_fpfh = preprocess_point_cloud(point_cloud, voxel_size) # means 5cm for this dataset
    frames.append(source_down)
    frames_fpfh.append(source_fpfh)
    
df_translation = pd.DataFrame({"frame_number":filenames})
df_rotation = pd.DataFrame({"frame_number":filenames})

now = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")


# rotations = [0,1,2]
# translations = [[0,0,0],[1,0,0],[2,0,0],[3,0,0],[3,3,0],[5,5,0],[10,10,0]]
translations = [[0,0,0],[1,0,0],[5,5,0],[10,10,0]][2:3]
for translation in translations:
    print(translation)
    i = 1
    translation_error = []
    rotation_error = []
    for idx, frame in enumerate(frames):
        print(str(idx) + " / " + str(len(frames)) + " ")

        temp_frame = copy.deepcopy(frame)
        temp_frame.translate(translation)
        ground_truth_rotation = rotate(temp_frame, 0,0,0)

        # evaluation = compute_icp_p2p(temp_frame, target)
        # evaluation = compute_icp_p2plane(temp_frame, target)
        evaluation = compute_global_registration_no_threshold(frame, target_down, frames_fpfh[idx], target_fpfh, voxel_size)
        # evaluation = compute_fast_global_registration(frame, target_down, frames_fpfh[idx], target_fpfh, voxel_size)

        # draw_point_clouds([target, temp_frame], transform(temp_frame, evaluation.transformation))
        translation_vector, rotation_matrix = decompose_transformation_matrix(evaluation.transformation)
        t_error = relative_translation_error(-np.array(translation), translation_vector)
        r_error = relative_rotation_error(rotation_matrix,ground_truth_rotation.T)
        translation_error.append(t_error)
        rotation_error.append(r_error)

        show_transformation(target, temp_frame, evaluation.transformation)

    df_translation[str(translation)] = translation_error
    df_rotation[str(translation)] = rotation_error
    df_translation.to_csv("C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\Daten\\rotated_and_translated\\100000000\\csv_results\\"+now+"_local_registration_translation_error_" + suffix + ".csv")
    df_rotation.to_csv("C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\Daten\\rotated_and_translated\\100000000\\csv_results\\"+now+"_local_registration_rotation_error_" + suffix + ".csv")



Beep(300, 200)
Beep(300, 200)
Beep(300, 700)

In [ ]:
Beep(300, 200)
Beep(300, 200)
Beep(300, 700)

In [ ]:
df_translation

In [ ]:
df_translation.plot()

In [ ]:
df_rotation

In [ ]:
df_rotation.plot()

In [ ]:
voxel_sizes = o3d.utility.DoubleVector([0.1, 0.05, 0.025])

# List of Convergence-Criteria for Multi-Scale ICP:
criteria_list = [
    treg.ICPConvergenceCriteria(relative_fitness=0.0001,
                                relative_rmse=0.0001,
                                max_iteration=20),
    treg.ICPConvergenceCriteria(0.00001, 0.00001, 15),
    treg.ICPConvergenceCriteria(0.000001, 0.000001, 10)
]

# `max_correspondence_distances` for Multi-Scale ICP (o3d.utility.DoubleVector):
max_correspondence_distances = o3d.utility.DoubleVector([0.3, 0.14, 0.07])

# Initial alignment or source to target transform.
init_source_to_target = o3d.core.Tensor.eye(4, o3d.core.Dtype.Float32)

# Select the `Estimation Method`, and `Robust Kernel` (for outlier-rejection).
estimation = treg.TransformationEstimationPointToPlane()

# Save iteration wise `fitness`, `inlier_rmse`, etc. to analyse and tune result.
callback_after_iteration = lambda loss_log_map : print("Iteration Index: {}, Scale Index: {}, Scale Iteration Index: {}, Fitness: {}, Inlier RMSE: {},".format(
    loss_log_map["iteration_index"].item(),
    loss_log_map["scale_index"].item(),
    loss_log_map["scale_iteration_index"].item(),
    loss_log_map["fitness"].item(),
    loss_log_map["inlier_rmse"].item()))


# Setting Verbosity to Debug, helps in fine-tuning the performance.
# o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)

s = time.time()

registration_ms_icp = treg.multi_scale_icp(source, target, voxel_sizes,
                                           criteria_list,
                                           max_correspondence_distances,
                                           init_source_to_target, estimation,
                                           callback_after_iteration)

ms_icp_time = time.time() - s
print("Time taken by Multi-Scale ICP: ", ms_icp_time)
print("Inlier Fitness: ", registration_ms_icp.fitness)
print("Inlier RMSE: ", registration_ms_icp.inlier_rmse)

draw_registration_result(source, target, registration_ms_icp.transformation)